In [2]:
!pip install torch
!pip install torchvision

In [2]:
import torch
import torchvision
from torchvision import transforms
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import GridSearchCV,cross_validate,train_test_split
import matplotlib.pyplot as plt

In [3]:
# converts a picture into normalized vector with values [0,1]
# more info here https://stackoverflow.com/questions/63746182/correct-way-of-normalizing-and-scaling-the-mnist-dataset
transformer = transforms.Compose([
                               torchvision.transforms.ToTensor(),
    transforms.Lambda(lambda x: torch.flatten(x)),
#                                transforms.Normalize(
#                                  (0.1307,), (0.3081,))
                             ])

In [4]:
train_set=torchvision.datasets.FashionMNIST('./files/fashion-mnist/', train=True, download=True,
                             transform=transformer)

test_set=torchvision.datasets.FashionMNIST('./files/fashion-mnist/', train=False, download=True,
                             transform=transformer)

batch_size_train= (int)(len(train_set)/10)
batch_size_test=len(test_set)

In [5]:
train_loader = torch.utils.data.DataLoader(
  train_set,
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  test_set,
  batch_size=batch_size_test, shuffle=True)

In [6]:
train_enumerated = enumerate(train_loader)
_, (train_x, train_y) = next(train_enumerated)

test_enumerated = enumerate(test_loader)
batch_idx, (test_x, test_y) = next(test_enumerated)

In [9]:
train_x.shape
test_x.shape

torch.Size([10000, 784])

In [12]:
from sklearn import svm, datasets
from sklearn.svm import SVC


In [31]:
clf = svm.SVC()
clf.fit(train_x,train_y)

SVC()

In [36]:
clf.predict(test_x)

array([3, 4, 7, ..., 4, 1, 9], dtype=int64)

In [34]:
clf.support_vectors_

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
from sklearn.model_selection import cross_val_score


In [12]:
scores = []

clf = svm.SVC()
    
with warnings.catch_warnings(record=True):
    score = cross_val_score(
            clf,
            train_x,
            train_y, 
            cv=5
        )
    scores.append(score.mean())

In [15]:
ovo = svm.SVC(decision_function_shape='ovo')
    
with warnings.catch_warnings(record=True):
    score = cross_val_score(
            ovo,
            train_x,
            train_y, 
            cv=5
        )
    scores.append(score.mean())

In [14]:
ovr = svm.SVC(decision_function_shape='ovr')
    
with warnings.catch_warnings(record=True):
    score = cross_val_score(
            ovr,
            train_x,
            train_y, 
            cv=5
        )
    scores.append(score.mean())

In [16]:
linear = svm.LinearSVC()
    
with warnings.catch_warnings(record=True):
    score = cross_val_score(
            linear,
            train_x,
            train_y, 
            cv=5
        )
    scores.append(score.mean())

LinearSVC also implements an alternative multi-class strategy, the so-called multi-class SVM formulated by Crammer and Singer, by using the option multi_class='crammer_singer'. In practice, one-vs-rest classification is usually preferred, since the results are mostly similar, but the runtime is significantly less. Using the Crammer and Singer formula is extremely slow for big datasets.

In [ ]:
fast_linear = svm.LinearSVC(multi_class='crammer_singer')
    
with warnings.catch_warnings(record=True):
    score = cross_val_score(
            fast_linear,
            train_x,
            train_y, 
            cv=5
        )
    scores.append(score.mean())

In [17]:
scores

[0.8535, 0.8535, 0.8535, 0.8019999999999999]

In [23]:
kernel_scores = []
gammas = [i/10 + 0.1 for i in range(10)]
cs = []

for number in gammas:
    clf = svm.SVC(kernel="rbf",gamma=number,C=1.0)
    
    with warnings.catch_warnings(record=True):
        score = cross_val_score(
                clf,
                train_x,
                train_y, 
                cv=5
            )
        kernel_scores.append(score.mean())

0.1
0.2
0.30000000000000004
0.4
0.5
0.6
0.7
0.7999999999999999
0.9
1.0


In [24]:
kernel_scores

[0.7671666666666667,
 0.5416666666666666,
 0.37700000000000006,
 0.25,
 0.195,
 0.16266666666666665,
 0.14583333333333334,
 0.13216666666666665,
 0.12250000000000001,
 0.11516666666666667]

In [16]:
kernel_scores = []
cs = [1/10*(i+1)  for i in range(10)]

for number in cs:
    clf = svm.SVC(kernel="rbf",gamma='auto',C=number)
    
    with warnings.catch_warnings(record=True):
        score = cross_val_score(
                clf,
                train_x,
                train_y, 
                cv=5
            )
        kernel_scores.append(score.mean())

In [17]:
kernel_scores

[0.7130000000000001,
 0.7435,
 0.76,
 0.7695000000000001,
 0.7758333333333334,
 0.7796666666666667,
 0.7838333333333333,
 0.7886666666666666,
 0.7925,
 0.7954999999999999]

In [26]:
auto_score = []

clf = svm.SVC(kernel="rbf",gamma='auto',C=1.0)

with warnings.catch_warnings(record=True):
    score = cross_val_score(
        clf,
        train_x,
        train_y, 
        cv=5
    )
    auto_score.append(score.mean())

In [14]:
model_lbls = [
              'lin', 
            ]

models = {
    'lin': {'name': 'Linear       ',
           'estimator': svm.LinearSVC(), 
           'param': [{}],
          }
}

scores = ['precision', 'recall']

In [10]:
def evaluate_models(models):
    results_short = {}
    for score in scores:
        print('='*40)
        print("# Tuning hyper-parameters for %s" % score)
        print()

        #'%s_macro' % score ## is a string formatting expression
        # the parameter after % is substituted in the string placeholder %s
        for m in model_lbls:
            print('-'*40)
            print("Trying model {}".format(models[m]['name']))
            clf = GridSearchCV(models[m]['estimator'], models[m]['param'], cv=5,
                               scoring='%s_macro' % score, 
    #                            iid = False, 
                               return_train_score = False,
                               n_jobs = 2, # this allows using multi-cores
                               )
            clf.fit(train_x, train_y)
            print_results(clf)
            results_short[m] = clf.best_score_
        print("Summary of results for {}".format(score))
        print("Estimator")
        for m in results_short.keys():
            print("{}\t - score: {:4.2}%".format(models[m]['name'], results_short[m]))

In [ ]:
evaluate_models(models)

# Tuning hyper-parameters for precision

----------------------------------------
Trying model Linear       


In [27]:
auto_score

[0.8053333333333332]

Стигаме до заключението, че гама трябва да е малко число близо до 0.

In [2]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

In [41]:
from sklearn.preprocessing import StandardScaler

In [42]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)

In [13]:
grid = [
#     {'kernel' : ['rbf'],
#     'C' : [0.001, 0.01, 0.1, 1 ,10],
#     'gamma' : [0.001, 0.01, 0.1, 1]},
    {'kernel' : ['linear'],
    'C' : [0.001, 0.01, 0.1, 1 ,10],
    }
]

search1 = GridSearchCV(SVC(),grid, cv=3)
search1.fit(test_x, test_y)

print(search1.best_params_)
print(search1.best_score_)


{'C': 0.1, 'kernel': 'linear'}
0.837099606781314


In [12]:
grid = [
    {'kernel' : ['rbf'],
    'C' : [0.001, 0.01, 0.1, 1 ,10,100],
    'gamma' : ["auto"]}
]

search = GridSearchCV(SVC(),grid, cv=5)
search.fit(test_x, test_y)

print(search.best_params_)
print(search.best_score_)

{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
0.8474


In [ ]:
auto_score = []

clf = svm.SVC(kernel="rbf",gamma='auto',C=1.0)

with warnings.catch_warnings(record=True):
    score = cross_val_score(
        clf,
        train_x,
        train_y, 
        cv=5
    )
    auto_score.append(score.mean())

In [45]:
from sklearn.metrics import accuracy_score


In [46]:
prediction = clf.predict(test_x)

In [48]:
acc = accuracy_score(test_y, prediction)
acc

0.8416